In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [3]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 225 µs (started: 2022-06-26 05:24:53 +00:00)


## Open Spark Session

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Reviews").getOrCreate()



time: 8.57 s (started: 2022-06-26 05:25:29 +00:00)


In [5]:
### Load Data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    



    
    
    
    The data is filtered to create a DataFrame or table where there isn’t a Vine review (5 pt)
    The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews (15 pt)


In [32]:
# There is a DataFrame or table for the vine_table data using one of three methods above (5 pt)
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"]).dropna()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1UUISQ1GKOJTI|          1|            0|          0|   N|                Y|
|R1HOJ9WE8VCVOD|          5|            9|          9|   N|                Y|
| RDNGVXMWQN2TN|          5|            0|          0|   N|                Y|
|R3OM9COQMVTDJ2|          2|            0|          0|   N|                Y|
|R3OFUQVR4Y80Q9|          4|            0|          0|   N|                N|
|R2GD25SBBGRKPO|          5|            0|          0|   N|                Y|
|R3RRSLYKVWH9WB|          5|            0|          0|   N|                Y|
|R2HMFAGJJU6NT3|          5|            0|          0|   N|                Y|
|R2QDRZATHBY4GY|          1|            0|          0|   N|                Y|
|R107946YZK57Q2|          2|            0|          0|   N|     

In [33]:
# The data is filtered to create a DataFrame or table where there are 20 or more total votes (5 pt)
from pyspark.sql.functions import col
votes_20 = vine_df.filter(col("total_votes") >= 20)
votes_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27XX9MG7OWSSM|          5|           49|         51|   N|                Y|
|R1I4IFDOW6IW9O|          3|           44|         47|   N|                Y|
|R2LZAV3VMLGPGS|          3|           32|         33|   N|                Y|
|R3FITR2BQI94RA|          5|           33|         38|   N|                Y|
| RWYBT1R3KPD2M|          4|          142|        163|   N|                Y|
| RE4023DS92U8H|          5|           26|         27|   N|                Y|
|R25TLB0EXZ7DJZ|          5|           45|         46|   N|                Y|
|R3BEQIBG1OQ7RT|          2|            3|         22|   N|                Y|
|R1SQKXYWU7VMW1|          2|           23|         24|   N|                Y|
| RUL6FPHYI15TA|          2|           62|         70|   N|     

In [34]:
# The data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50% (5 pt)
percent50_votes_df = votes_20.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
percent50_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R27XX9MG7OWSSM|          5|           49|         51|   N|                Y|0.9607843137254902|
|R1I4IFDOW6IW9O|          3|           44|         47|   N|                Y|0.9361702127659575|
|R2LZAV3VMLGPGS|          3|           32|         33|   N|                Y|0.9696969696969697|
|R3FITR2BQI94RA|          5|           33|         38|   N|                Y| 0.868421052631579|
| RWYBT1R3KPD2M|          4|          142|        163|   N|                Y|0.8711656441717791|
| RE4023DS92U8H|          5|           26|         27|   N|                Y|0.9629629629629629|
|R25TLB0EXZ7DJZ|          5|           45|         46|   N|                Y|0.9782608695652174|
|R1SQKXYWU7VMW1|          2|  

In [35]:
# The data is filtered to create a DataFrame or table where there is a Vine review (5 pt)
vine_review = percent50_votes_df.filter(col("vine") == "Y")
vine_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2W1OMYJERBXJ1|          5|           20|         23|   Y|                N|0.8695652173913043|
| R31VJCBBKU13Y|          5|           24|         27|   Y|                N|0.8888888888888888|
|R350JNDV0INC11|          4|          388|        398|   Y|                Y|0.9748743718592965|
|R2NJA2I5C9Y0FR|          3|           63|         73|   Y|                N| 0.863013698630137|
|R296RSG25RD1IX|          3|           46|         51|   Y|                N|0.9019607843137255|
|R3IJKA4RI56TFT|          4|           25|         30|   Y|                N|0.8333333333333334|
|R1EU2LNHAWWPFI|          5|           19|         24|   Y|                N|0.7916666666666666|
|R164L6F7QRHL8A|          5|  

In [36]:
# The data is filtered to create a DataFrame or table where there isn't a Vine review (5 pt)
non_vine_review = percent50_votes_df.filter(col("vine") == "N")
non_vine_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R27XX9MG7OWSSM|          5|           49|         51|   N|                Y|0.9607843137254902|
|R1I4IFDOW6IW9O|          3|           44|         47|   N|                Y|0.9361702127659575|
|R2LZAV3VMLGPGS|          3|           32|         33|   N|                Y|0.9696969696969697|
|R3FITR2BQI94RA|          5|           33|         38|   N|                Y| 0.868421052631579|
| RWYBT1R3KPD2M|          4|          142|        163|   N|                Y|0.8711656441717791|
| RE4023DS92U8H|          5|           26|         27|   N|                Y|0.9629629629629629|
|R25TLB0EXZ7DJZ|          5|           45|         46|   N|                Y|0.9782608695652174|
|R1SQKXYWU7VMW1|          2|  

In [37]:
# The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews (15 pt)

## All Vine Reviews
total_vine_reviews = vine_review.count()
vine_5star = vine_review.filter(vine_review['star_rating']== 5).count()
percent_5star_vine = float(vine_5star) / float(total_vine_reviews)
print(f'VINE REVIEW METRICS')
print("---------------------------------------------------------")
print(f'Total Vine review             : {total_vine_reviews}')
print(f'Total Vine 5 star review      : {vine_5star}')
print(f'Percent Vine 5 star reviews   : {percent_5star_vine:0.2f}')
print("---------------------------------------------------------")


## All Non Vine Reviews
total_non_vine_reviews = non_vine_review.count()
non_vine_5star = non_vine_review.filter(non_vine_review['star_rating']== 5).count()
percent_5star_non_vine = float(non_vine_5star) / float(total_non_vine_reviews)
print(f'NON VINE REVIEW METRICS')
print("---------------------------------------------------------")
print(f'Total Non Vine review             : {total_non_vine_reviews}')
print(f'Total Non Vine 5 star review      : {non_vine_5star}')
print(f'Percent Non Vine 5 star reviews   : {percent_5star_non_vine:0.2f}')
print("---------------------------------------------------------")

VINE REVIEW METRICS
---------------------------------------------------------
Total Vine review          : 1448
Total Vine 5 star review   : 647
Percent Vine 5 star revi   : 0.45
---------------------------------------------------------
NON VINE REVIEW METRICS
---------------------------------------------------------
Total Non Vine review          : 90768
Total Non Vine 5 star review   : 44104
Percent Non Vine 5 star revi   : 0.49
---------------------------------------------------------
time: 2min 15s (started: 2022-06-26 06:23:51 +00:00)
